In [10]:
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

In [11]:
# imports
import sys
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import Data.data_provider as dp
import core as cal
from estimators.IR_RF_estimator import IR_RF

In [12]:
# params
seed = 0
runs = 100
calib_methods = ["RF", "Platt" , "ISO", "Rank", "CRF", "VA", "Beta", "Elkan", "tlr"]
metrics = ["acc", "auc", "brier", "ece"]
# data_list = ["spambase", "climate", "QSAR", "bank", "climate", "parkinsons", "vertebral", "ionosphere", "diabetes", "breast", "blod"]
data_list = ["spambase", "climate"]
# data_list = ["synthetic1"] # , "synthetic2"

params = {
    "n_estimators": 100,
    "oob": False,
}

In [13]:
results_dict = {}
for data_name in data_list:
    # Data
    X, y = dp.load_data(data_name, "../../")
    # X, y, tp = dp.make_classification_gaussian_with_true_prob(30, 4, 0)
    # split to train calib test

    data_dict = {}
    for seed in range(runs):
        data = cal.split_train_calib_test(data_name, X, y, 0.3, 0.5, seed)
        # train model
        irrf = IR_RF(n_estimators=params["n_estimators"], oob_score=params["oob"], random_state=seed)
        irrf.fit(data["x_train"], data["y_train"])

        # calibration
        res = cal.calibration(irrf, data, calib_methods, metrics)
        data_dict = cal.update_runs(data_dict, res)
    results_dict.update(data_dict)

table = cal.mean_and_ranking_table(results_dict, metrics, calib_methods, data_list)

--------------------------------- acc
                RF     Platt       ISO      Rank       CRF        VA      Beta     Elkan       tlr
Data                                                                                              
spambase  0.924692  0.923244  0.923968  0.929037  0.924692  0.385228  0.923968  0.924692  0.923244
climate   0.922222  0.927160  0.928395  0.881481  0.922222  0.919753  0.927160  0.922222  0.919753
Mean      0.923457  0.925202  0.926182  0.905259  0.923457  0.652491  0.925564  0.923457  0.921499
Rank      4.000000  5.000000  3.250000  5.000000  4.000000  8.250000  4.000000  4.000000  7.500000
--------------------------------- auc
                RF     Platt       ISO      Rank       CRF        VA      Beta     Elkan       tlr
Data                                                                                              
spambase  0.972836  0.972836  0.972562  0.971828  0.972836  0.972683  0.972836  0.972836  0.968025
climate   0.820593  0.820593  0.8